# Project 2: Strain Heterogeneity Project

## Description

This project is to analyze the effect of HGT on strain heterogeneity on the binning of the bacterial genomes from metagenome.

The hypothesis is that high HGT results causes gene-specific sweeps that results in high strain heterogeneity. Therefore, taxa that exhibits high hgt signals should have high strain heterogeneity and lower quality bins.

## <Insert Pipeline>


## Strain Analysis

Here we will check the heterogeneity per taxa and per bin identified in metagenomes. The metagenomes that will be used are those from 2020-2022 GOWN samples.

First we need to identify the amount of heterogeneity per taxa studied. This is measured by *MiDas2/3* and directly identifying strains using *InStrain* Pipeline.


### InStrain Pipeline

Here let us run the InStrain Pipeline. First we need to build a genome database to which the metagenomes will be compared too.

<ins>**Dereplicate**</ins> the intraspecific species genomes with mash at ANI: 98% (I am looking at 99.2% which equivalent to 1.2 bp difference in a 150 long read). A 98% similarity is equivalent to read similarity of 3bp difference in a 150 bp long read.

Dereplication was done in three modes:
1. Bin Only
2. GTDB Only
3. GTDB + Bin

To dereplicate bins only, the following code was run from `/home/gamaliellysandergl.c/p2_workstation/drep`:

In [ ]:
%%bash
# drep{1}.slurm
#!/bin/bash
# ---------------------------------------------------------------------
# An example PBS script for running a job on a compute cluster
# ---------------------------------------------------------------------
#SBATCH --job-name=dRepBin98
#SBATCH --nodes=1
#SBATCH --cpus-per-task=16
#SBATCH --mem=240gb
#SBATCH --time=7-00:00:00

# ---------------------------------------------------------------------
echo "Starting run at: `date`"
# ---------------------------------------------------------------------

source ~/software/miniforge3/etc/profile.d/conda.sh 
conda activate instrain

workdir='/home/gamaliellysandergl.c/p2_workstation'

### 01 - Preparing for Drep ###
# Import the reference genome from the GTDB downloaded database.
# This version uses the GTDB R214 release
# python instrain_ref_fetch.py instrain_taxa_list.txt
# bash instrain_bin_fetch.sh instrain_bin_list.txt
# echo "inStrain GTDB reference database concatenate at `date`"

### 02 - Drep Run ###
dRep dereplicate BinGenomeSet -g /home/gamaliellysandergl.c/p2_workstation/bins/*.fa \
--completeness 50 \
--checkM_method taxonomy_wf \
--S_algorithm fastANI --multiround_primary_clustering \
--clusterAlg average \
-ms 10000 \
-pa 0.9 -sa 0.98 \
-nc 0.30 -cm larger -p 16 \
-d


The results can be find in: `/home/gamaliellysandergl.c/p2_workstation/drep/BinGenomeSet`

A different version which combines the GTDB Genome Set and Bin Genome Set into one is made by `drep3.slurm`. The output folder is named as `MergedGenomeSet`. The code is here:

In [ ]:
%%bash
#drep3.slurm

## <insert SBATCH parameters here>

# --------------------------------------------------------------------
echo "Starting run at: `date`"
# --------------------------------------------------------------------

source ~/software/miniforge3/etc/profile.d/conda.sh 
conda activate instrain

workdir='/work/ebg_lab/referenceDatabases/dRep'
scratch=/scratch/$SLURM_JOBID


### 01 - Preparing for Drep ###
# Import the reference genome from the GTDB downloaded database.
# This version uses the GTDB R214 release
# python instrain_ref_fetch.py instrain_taxa_list.txt
# bash instrain_bin_fetch.sh instrain_bin_list.txt
# echo "inStrain GTDB reference database concatenate at `date`"

# Decompress *.gz files in scratch
while IFS= read -r filepath; do
filename=${filepath##*/} # same as $(basename "$filepath")
prefix=$(echo "$filename" | rev | cut -d'.' -f3- | rev)
newname="${prefix}.fa"

cp ${filepath} ${scratch}/${filename}
gzip -d "${scratch}/${filename}"

mv ${scratch}/${filename%.*} ${scratch}/${newname}

done < ref_genome_path.txt

echo "`ls ${scratch}`"

# Move Bins to scratch
cp /home/gamaliellysandergl.c/p2_workstation/bins/*.fa ${scratch}


### 02 - Drep Run ###
dRep dereplicate ${scratch}/MergedGenomeSet -g ${scratch}/*.fa \
--completeness 50 \
--checkM_method taxonomy_wf \
--S_algorithm fastANI --multiround_primary_clustering \
--clusterAlg average \
-ms 10000 \
-pa 0.9 -sa 0.98 \
-nc 0.30 -cm larger -p 16 \
-d


The outputs for unique genomes for all Genome Sets can be found in `drep/*GenomeSet/dereplicated_genomes`. The content of this folder are same format and name as the input. 

<ins>**Concatenation**</ins> of these genomes into a single fasta is needed to make the reference for inStrain. Unfortunately, Bin Genomes sometimes have the same headers between different fasta files so it can create errors during inStrain. Therefore, prior to concatenation, we will rename the headers by prefixing the headers with the genome/fasta filename (i.g. *>O10.maxbin.2.fa--K141_123124*). The code for concatenation is down below:


In [ ]:
%%bash
#drep_concatenate.slurm

#!/bin/bash
# ---------------------------------------------------------------------
# An example PBS script for running a job on a compute cluster
# ---------------------------------------------------------------------
#SBATCH --job-name=dRepcat
#SBATCH --nodes=1
#SBATCH --cpus-per-task=4
#SBATCH --mem=24gb
#SBATCH --time=7-00:00:00
# ---------------------------------------------------------------------
echo "Starting run at: `date`"
# ---------------------------------------------------------------------

source ~/software/miniforge3/etc/profile.d/conda.sh 
conda activate instrain

for i in `ls -d *GenomeSet`
do
workdir=${i}/dereplicated_genomes

    for j in `ls ${workdir}/*.fa`
    do
    filename=${j##*/}
    sed -i "s/^>/>${filename}--/g" ${j}
    done #j

cat ${workdir}/*.fa > ${i}/${i}_concatenated.fa

done #i

The concatenated genome will be a single file inside the Genome Set folders. 

Now that the reference genomes has been dereplicated and concatenated, we will process the reference fasta file through <ins>**parse_stb**</ins>, building <ins>**bowtie index**</ins>, creating a bam file using <ins>**bowtie**</ins> it self, and predicting genes using <ins>**prodigal**</ins> .

<ins>**Parsing**</ins> the input genomes into a scaffold-to-bin (stb) file allows back tracking of the contigs to its origin and needed by inStrain. The `drep_parse_stb.slurm` uses the `parse_stb.py` that comes along drep installation to create the stb file:

In [ ]:

%%bash
#!/bin/bash
# ---------------------------------------------------------------------
# An example PBS script for running a job on a compute cluster
# ---------------------------------------------------------------------
#SBATCH --job-name=dRepstb
#SBATCH --nodes=1
#SBATCH --cpus-per-task=4
#SBATCH --mem=24gb
#SBATCH --time=7-00:00:00

# ---------------------------------------------------------------------
echo "Starting run at: `date`"
# ---------------------------------------------------------------------

source ~/software/miniforge3/etc/profile.d/conda.sh 
conda activate instrain

for i in `ls -d *GenomeSet`
do
echo $i
parse_stb.py --reverse -f ${i}/dereplicated_genomes/*.fa \
-o ${i}_parse.stb 

mv ${i}_parse.stb ${i}/

done

The stb files are moved into individual Genome Set folders with the filename is the Genome Set name: `*GenomeSet_parse.stb`.

<ins>**Bowtie index**</ins> was build using bowtie2-build default options. Unlike the prior `slurm` files, bowtie indexing were done inside the `instrain2` folder which is adjacent to `drep` folder. The `instrain2` folder contains the `bowtie` folder and inside `bowtie` is `bowtie-build` which contains the indices. The script is here:

In [ ]:
%%bash
#instrain_1_bowtie.slurm

#!/bin/bash
# ---------------------------------------------------------------------
# An example PBS script for running a job on a compute cluster
# ---------------------------------------------------------------------
#SBATCH --job-name=instBT2build
#SBATCH --nodes=1
#SBATCH --cpus-per-task=16
#SBATCH --mem=64gb
#SBATCH --time=7-00:00:00

# ---------------------------------------------------------------------
echo "Starting run at: `date`"
# ---------------------------------------------------------------------

source ~/software/miniforge3/etc/profile.d/conda.sh 
conda activate bowtie

workdir=${PWD}/bowtie/bowtie2-build
mkdir -p ${workdir}
cd ${workdir}

for i in `ls -d /home/gamaliellysandergl.c/p2_workstation/drep/*Set`
do
filename=${i##*/}
echo ${filename}
bowtie2-build ${i}/${filename}_concatenated.fa ${filename}_concatenated.fa 
done

After the indices are built, the main <ins>**bowtie**</ins> run will be run. The `*.sam` files are located inside the `bowtie` folder with the name `<Reference GenomeSet>_v_<Metagenome Reads>.sam`. The code for it is:

In [ ]:
%%bash
#instrain_2_bowtie.slurm

#!/bin/bash
# ---------------------------------------------------------------------
# An example PBS script for running a job on a compute cluster
# ---------------------------------------------------------------------
#SBATCH --job-name=inStrBT2
#SBATCH --nodes=1
#SBATCH --cpus-per-task=20
#SBATCH --mem=80gb
#SBATCH --time=7-00:00:00

# ---------------------------------------------------------------------
echo "Starting run at: `date`"
# ---------------------------------------------------------------------

source ~/software/miniforge3/etc/profile.d/conda.sh 
conda activate bowtie

# PWD run inside instrain2

workdir=${PWD}/bowtie/bowtie2-build
outputdir=${PWD}/bowtie
mkdir -p ${workdir}


# Fetch the metagenomes

for j in `ls -d ~/p2_workstation/instrain2/metagenomes/AOB*/`
do
metagenome=$(echo $j | rev | cut -d/ -f2 | rev)
R1="`ls ${j}/*qc.R1.fastq`"
R2="`ls ${j}/*qc.R2.fastq`"

# Fetch the references
for i in `ls -d /home/gamaliellysandergl.c/p2_workstation/drep/*Set`
do
filename=${i##*/} #filename is either BinGenomeSet, GTDBGenomeSet, or MergedGenomeSet
output_file="${filename}_v_${metagenome}"
echo "# Running: ${output_file}"

bowtie2 -p $SLURM_CPUS_PER_TASK \
-x ${workdir}/${filename}_concatenated.fa \
-1 ${R1} -2 ${R2} > \
${outputdir}/${output_file}.sam
done

done #j


On the side, <ins>**Prodigal**</ins> can be run simultaneously with the *bowtie* slurm files without dependencies. The prodigal files will be in the `drep/*GenomeSet/prodigal`  folder outside of `instrain` folder. The it should contain `*genes.fna` and `*genes.faa` files. The script:

In [ ]:
%%bash
#instrain_3_prodigal.slurm

#!/bin/bash
# ---------------------------------------------------------------------
# An example PBS script for running a job on a compute cluster
# ---------------------------------------------------------------------
#SBATCH --job-name=instrprodigal
#SBATCH --nodes=1
#SBATCH --cpus-per-task=8
#SBATCH --mem=24gb
#SBATCH --time=7-00:00:00

# ---------------------------------------------------------------------
echo "Starting run at: `date`"
# ---------------------------------------------------------------------

source ~/software/miniforge3/etc/profile.d/conda.sh 
conda activate instrain

for i in `ls -d ~/p2_workstation/drep/*Set`
do
GenomeSet=${i##*/}
echo ${GenomeSet}

mkdir -p ${i}/prodigal

#echo " \
prodigal -i ${i}/${GenomeSet}_concatenated.fa \
-d ${i}/prodigal/${GenomeSet}_concatenated.fa.genes.fna \
-a ${i}/prodigal/${GenomeSet}_concatenated.fa.genes.faa
#"
done

With all the above preparation done, we now run the `inStrain profile` command to get the initial IS profile output of InStrain. The command we use is this:

In [ ]:
%%bash
#instrain_4_profile.slurm

#!/bin/bash
# ---------------------------------------------------------------------
# An example PBS script for running a job on a compute cluster
# ---------------------------------------------------------------------
#SBATCH --job-name=inStrProfile
#SBATCH --nodes=1
#SBATCH --cpus-per-task=32
#SBATCH --mem=128gb
#SBATCH --time=7-00:00:00

# ---------------------------------------------------------------------
echo "Starting run at: `date`"
# ---------------------------------------------------------------------

source ~/software/miniforge3/etc/profile.d/conda.sh 
conda activate instrain

# PWD run inside instrain2
samdir=${PWD}/bowtie
profile_dir=${PWD}/profile
mkdir -p ${profile_dir}

# Fetch the metagenomes

for j in `ls -d ~/p2_workstation/instrain2/metagenomes/AOB*/`
do
metagenome=$(echo $j | rev | cut -d/ -f2 | rev)

# Fetch the references
for i in `ls -d /home/gamaliellysandergl.c/p2_workstation/drep/*Set`
do
GenomeSet=${i##*/} #GenomeSet is either BinGenomeSet, GTDBGenomeSet, or MergedGenomeSet
bam_file="${GenomeSet}_v_${metagenome}"
fasta_file="`ls ${i}/*concatenated.fa`"
genes="`ls ${i}/prodigal/*genes.fna`"
parse_stb="`ls ${i}/*.stb`"
echo "# Running: ${bam_file}"

#echo "\
inStrain profile \
${samdir}/${bam_file}.sam \
${fasta_file} \
-p $SLURM_CPUS_PER_TASK \
-o ${profile_dir}/${bam_file}.IS \
-g ${genes} \
-s ${parse_stb}
#"

done

done #j

Each sam files was converted to bam files. The output profile is created per each bam file. Thus, the amount of profile created depends on `No. of GenomeSet * No. of Metagenomes`. The IS profiles output are one per combination mentioned. It is a folder and more details on its output can be found on this [inStrain documentation](https://instrain.readthedocs.io/en/latest/).

#### Analysis of inStrain profile